In [1]:
import cv2 
import numpy as np
import math
image = cv2.imread('road.jpg')

img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

gray_gaussain = cv2.GaussianBlur(img_gray,(5,5),0)

gray_canny = cv2.Canny(gray_gaussain,75,175)
cv2.imshow('new',gray_canny)
cv2.waitKey(0)

#FORMATION OF MASKING LAYER
mask = np.zeros_like(gray_canny)

ignore_mask_color = 255 

imshape = image.shape
    
apex_left = [400, 355]
apex_right = [imshape[1] - 380, 355]
bottom_left = [175, imshape[0]]
bottom_right = [imshape[1] - 39,imshape[0]]
    
vertices = np.array([[(bottom_left[0],bottom_left[1]), \
                          (apex_left[0],apex_left[1]), \
                          (apex_right[0], apex_right[1]), \
                          (bottom_right[0], bottom_right[1])]], dtype=np.int32)

ysize = image.shape[0]
xsize = image.shape[1]


cv2.fillPoly(mask, vertices, ignore_mask_color)

masked_image = cv2.bitwise_and(img_gray, mask)
masked_edge = cv2.bitwise_and(gray_canny,mask) # main image to take out the hough lines 


lines = cv2.HoughLinesP(masked_edge, 1,  np.pi / 180, 50, None,25,
                                  125)

width , height = masked_edge.shape[:2]
line_img = np.zeros((width,height, 3), dtype=np.uint8)  
draw_lines(line_img,lines)

new_img = np.zeros((width,height),dtype = np.uint8)
    
print lines



# LINE IS DRAWN
for x1, y1, x2, y2 in lines[0]:
    cv2.line(new_img, (x1, y1), (x2, y2),(255,127,0), 3)
    

#APPLYING WEIGHT IMAGE
new = cv2.addWeighted(new_img,0.8,img_gray,1.,0.)

#co-ordinates of the turning lines

h1,k1,p1,q1 = 591,441,750,441
h2,k2,p2,q2 = 194,441,374,441

#turing line equations
cv2.line(new, (h1,k1), (p1,q1), (0,0,0), 5)
cv2.line(new, (h2,k2), (p2,q2), (0,0,0), 5)
cv2.imshow('new img',new_img)
cv2.imshow('new',new)

def imshow():
   
    cv2.imshow('new mask image',mask)
    cv2.imshow('masked image',masked_image)
    cv2.imshow('masked image edge',masked_edge)

    
imshow()

    
    
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'draw_lines' is not defined

In [4]:
prev_left_x_min = None
prev_left_x_max = None
prev_right_x_min = None
prev_right_x_max = None

In [5]:
def draw_lines(img, lines, color=[0, 255, 0], thickness=11):
   
    x1_values = lines[:, :, 0]
    y1_values = lines[:, :, 1]
    x2_values = lines[:, :, 2]
    y2_values = lines[:, :, 3]


    global prev_left_x_min
    global prev_left_x_max
    global prev_right_x_min
    global prev_right_x_max

    y_max = max(np.amax(y1_values), np.amax(y2_values))
    y_min = max(np.amin(y1_values), np.amin(y2_values))
    left_slopes = []
    right_slopes = []

    right_lane = []
    left_lane = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)

#             if slope < 0:  #left lane has negative slope
            if (-0.9 < slope < -0.6):
                left_slopes.append(slope)
                left_lane.append(line)
            elif 0.45 < slope < 0.7: # right lane positive slope
                right_slopes.append(slope)
                right_lane.append(line)

    left_slopes_arr = np.array(left_slopes)
    right_slopes_arr = np.array(right_slopes)

    left_lines_arr = np.array(left_lane)
    right_lines_arr = np.array(right_lane)

    # Find average slopes
    left_lane_slope_average = np.mean(left_slopes_arr)  # type numpy.float64
    right_lane_slope_average = np.mean(right_slopes_arr)

    y_max = int(y_max)
    y_min = int(y_min)

    if len(left_lines_arr) > 0:
        # Compute Left lane intercept:
        y_intercept_left_lane_average = np.mean(left_lines_arr[:, :, 1] - (left_lines_arr[:, :, 0] * left_lane_slope_average))

        # Compute Left lane Xmax and Xmin:
        x_max_left = int((y_max - y_intercept_left_lane_average) / left_lane_slope_average)
        x_min_left = int((y_min - y_intercept_left_lane_average) / left_lane_slope_average)
            

        cv2.line(img, (x_max_left, y_max), (x_min_left, y_min), color, thickness)
        # Save last coordinates to keep line going
        
        prev_left_x_max = x_max_left
        prev_left_x_min = x_min_left

    else:
        
        if prev_left_x_max is not None:
            print('prev_left_x_max is not None')
            cv2.line(img, (prev_left_x_max, y_max),(prev_left_x_min, y_min), color, thickness)
            cv2.imshow('new try',img)
        else:
            print('nothing')
            
    if len(right_lines_arr) > 0:
        # Compute Right Lane Intercept:
        y_intercept_right_lane_average = np.mean(right_lines_arr[:, :, 1] - (right_lines_arr[:, :, 0] * right_lane_slope_average))

        # Compute Right lane Xmax and Xmin:
        x_max_right = int((y_max - y_intercept_right_lane_average) / right_lane_slope_average)
        x_min_right = int((y_min - y_intercept_right_lane_average) / right_lane_slope_average)

        cv2.line(img, (x_max_right, y_max), (x_min_right, y_min), color, thickness)
        cv2.inshow('right',img)
        prev_right_x_max = x_max_right
        prev_right_x_min = x_min_right
    else:
        if prev_right_x_max is not None:
            print('prev_right_x_max is not NONE')
#         print("right_lines_arr is empty. Not drawing anything as there are no lines to choose from.")
            cv2.line(img, (prev_right_x_max, y_max), (prev_right_x_min, y_min), color, thickness)
            cv2.imshow('new try 2',img)
        else:
            
            print('nothing right side empty')


In [21]:
# Test on solidWhite
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'submit-solidWhiteRightx2.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

nothing
nothing right side empty
[[565 355 870 538]
 [564 355 869 539]
 [579 367 774 493]
 [621 393 847 539]
 [239 487 366 391]]
right
[565 355 870 538]
[564 355 869 539]
[579 367 774 493]
[621 393 847 539]
left
[239 487 366 391]
new list left
[array([239, 487, 366, 391])]
new list right
[array([564, 355, 869, 539]), array([564, 355, 869, 539]), array([621, 393, 847, 539]), array([565, 355, 870, 538])]
new single line list left: [239 487 366 391]
new single line list right: [564 355 869 539]
left intercept point: [ 441.          299.85416667]
right intercept point: [ 441.          706.55434783]
TURN LEFT
[MoviePy] >>>> Building video submit-solidWhiteRightx2.mp4
[MoviePy] Writing video submit-solidWhiteRightx2.mp4


  0%|          | 0/222 [00:00<?, ?it/s]

nothing
nothing right side empty
[[558 356 861 538]
 [638 407 847 538]
 [557 355 860 537]
 [626 399 837 531]]
right
[558 356 861 538]
[638 407 847 538]
[557 355 860 537]
[626 399 837 531]
left
new list left
[0, 0, 0, 0]
new list right
[array([558, 356, 861, 538]), array([557, 355, 860, 537]), array([558, 356, 861, 538])]
new single line list left: 0
new single line list right: [558 356 861 538]
left intercept point: [   441.  49000.]
right intercept point: [ 441.          699.51098901]
TURN RIGHT


  0%|          | 1/222 [00:00<00:34,  6.37it/s]

nothing
nothing right side empty
[[601 381 845 539]
 [563 355 868 538]
 [565 355 756 470]
 [630 399 843 537]
 [583 370 770 491]
 [238 488 402 360]
 [760 474 867 539]]
right
[601 381 845 539]
[563 355 868 538]
[565 355 756 470]
[630 399 843 537]
[583 370 770 491]
left
[238 488 402 360]
new list left
[array([238, 488, 402, 360])]
new list right
[array([563, 355, 868, 538]), array([563, 355, 868, 538]), array([630, 399, 843, 537]), array([630, 399, 843, 537]), array([601, 381, 845, 539])]
new single line list left: [238 488 402 360]
new single line list right: [563 355 868 538]
left intercept point: [ 441.       298.21875]
right intercept point: [ 441.          706.33333333]
TURN LEFT


...

nothing
nothing right side empty
[[593 370 896 538]
 [618 389 875 538]
 [196 538 407 367]
 [561 355 761 471]
 [567 355 895 537]
 [194 524 405 365]
 [206 514 385 379]]
right
[593 370 896 538]
[618 389 875 538]
[561 355 761 471]
[567 355 895 537]
left
[196 538 407 367]
[194 524 405 365]
[206 514 385 379]
new list left
[array([196, 538, 407, 367]), array([194, 524, 405, 365]), array([196, 538, 407, 367])]
new list right
[array([593, 370, 896, 538]), array([567, 355, 895, 537]), array([593, 370, 896, 538])]
new single line list left: [196 538 407 367]
new single line list right: [593 370 896 538]
left intercept point: [ 441.          315.69005848]
right intercept point: [ 441.          721.05357143]
TURN RIGHT


 99%|█████████▉| 220/222 [00:34<00:00,  6.45it/s]

nothing
nothing right side empty
[[597 372 896 538]
 [591 373 876 538]
 [561 355 806 497]
 [568 355 642 396]
 [676 423 875 538]
 [195 538 403 370]
 [571 357 897 538]]
right
[597 372 896 538]
[591 373 876 538]
[561 355 806 497]
[568 355 642 396]
[676 423 875 538]
[571 357 897 538]
left
[195 538 403 370]
new list left
[array([195, 538, 403, 370])]
new list right
[array([597, 372, 896, 538]), array([591, 373, 876, 538]), array([561, 355, 806, 497]), array([676, 423, 875, 538]), array([597, 372, 896, 538])]
new single line list left: [195 538 403 370]
new single line list right: [597 372 896 538]
left intercept point: [ 441.         315.0952381]
right intercept point: [ 441.          721.28313253]
TURN RIGHT


100%|█████████▉| 221/222 [00:34<00:00,  6.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: submit-solidWhiteRightx2.mp4 

Wall time: 35.7 s


In [18]:
import numpy as np
import math
import cv2
import json

car_classifier = cv2.CascadeClassifier('haarcascade_car.xml')
def process_image(image):
    
    left = []
    right = []
    new_line_left = []
    new_line_right = []
    
    

    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    
    gray_gaussain = cv2.GaussianBlur(img_gray,(5,5),0)

    gray_canny = cv2.Canny(gray_gaussain,75,175)

    mask = np.zeros_like(gray_canny)

    ignore_mask_color = 255 
    
    imshape = image.shape
    
    apex_left = [400, 355]
    apex_right = [imshape[1] - 380, 355]
    bottom_left = [175, imshape[0]]
    bottom_right = [imshape[1] - 39,imshape[0]]
    
    vertices = np.array([[(bottom_left[0],bottom_left[1]), \
                          (apex_left[0],apex_left[1]), \
                          (apex_right[0], apex_right[1]), \
                          (bottom_right[0], bottom_right[1])]], dtype=np.int32)

    ysize = image.shape[0]
    xsize = image.shape[1]


    cv2.fillPoly(mask, vertices, ignore_mask_color)

    masked_image = cv2.bitwise_and(img_gray, mask)
    masked_edge = cv2.bitwise_and(gray_canny,mask) # main image to take out the hough lines 


    lines = cv2.HoughLinesP(masked_edge, 1,  np.pi / 180, 50, None,25,
                                  125)

    width , height = masked_edge.shape[:2]
    line_img = np.zeros((width,height, 3), dtype=np.uint8)  
    draw_lines(line_img,lines)

    new_img = np.zeros((width,height),dtype = np.uint8)
    
    #print (lines)
# LINE IS DRAWN
    for x1, y1, x2, y2 in lines[0]:
        cv2.line(new_img, (x1, y1), (x2, y2),(255,127,0), 3)

#APPLYING WEIGHT IMAGE
    new = cv2.addWeighted(new_img,0.8,img_gray,1.,0.)
    
    h1,k1,p1,q1 = 591,441,750,441
    h2,k2,p2,q2 = 194,441,374,441

#turing line equations
    cv2.line(new, (h1,k1), (p1,q1), (0,0,255), 5)
    cv2.line(new, (h2,k2), (p2,q2), (0,0,255), 5)
    
    #--------------------------------------------------------------------------------

    
    line_list = lines[0]
    print line_list 
    
    for i in range (len(line_list)):
        if(i< len(line_list)):
            l1 = line_list[i]
            if(l1[0]>=140 and l1[0]<= 446):
                left.append(l1)
            elif (l1[0]>=515 and l1[0]<=754):
                right.append(l1)
    
            
            
    def print_sorted_points():            
        print 'right'            
        for i in right :
            print i

        print 'left'
        for i in left :
            print i

    print_sorted_points()   
    
    if(len(left) !=0): 
        for i in range (len(left)):
            if(i< len(left)-1):
                l_1 = left[i]
                l_2 = left[i+1]
                if(l_1[0] - l_2[0] > 50):
                    pass
                else:
                    d_l_1 = math.sqrt(pow(l_1[2]-l_1[0],2)+pow(l_1[3]-l_1[1],2))
                    d_l_2 = math.sqrt(pow(l_2[2]-l_2[0],2)+pow(l_2[3]-l_2[1],2))
                    if(d_l_1 >= d_l_2):
                        new_line_left.append(l_1)
                
                    else:
                        new_line_left.append(l_2)
            else:
                new_line_left.append(left[0])
            
    elif(len(left) == 0):
        new_line_left = [0,0,0,0]
            

    print 'new list left'            
    print new_line_left
    if(len(right) !=0):
        for i in range (len(right)):
            if(i< len(right)-1):#in the case when lenght of the list is '1' and '0' it fails
                l_1 = right[i]
                l_2 = right[i+1]
                if(l_1[0] - l_2[0] > 50):
                    pass
                else:
                    d_l_1 = math.sqrt(pow(l_1[2]-l_1[0],2)+pow(l_1[3]-l_1[1],2))
                    d_l_2 = math.sqrt(pow(l_2[2]-l_2[0],2)+pow(l_2[3]-l_2[1],2))
                    if(d_l_1 >= d_l_2):
                        new_line_right.append(l_1)
                
                    else:
                        new_line_right.append(l_2)
            else:
                 new_line_right.append(right[0])
    
    elif(len(right) == 0):
        new_line_right = [0,0,0,0]
   
    
    
    
    print 'new list right'            
    print new_line_right




#-------------------------------------------------------------------------------------------------------------------
#drawing lines on the image with the sorted line co-ordinates           
#selecting one line
    left_1 = new_line_left[0]
    right_1 = new_line_right[0]
    print 'new single line list left: '+str(left_1)
    print 'new single line list right: '+str(right_1)



#co-ordinates of the equation
    if(str(left_1) == '0' and str(right_1) != 0):
        x1,y1,x2,y2= 0,0,0,0
        a1,b1,a2,b2 = right_1
    elif(str(right_1) == '0' and str(left_1) != 0):
        x1,y1,x2,y2= left_1
        a1,b1,a2,b2 = 0,0,0,0
    else:
        x1,y1,x2,y2= left_1
        a1,b1,a2,b2 = right_1
        

    cv2.line(image, (x1,y1), (x2,y2), (255,127,0), 5)
    cv2.line(image, (a1,b1), (a2,b2), (255,127,0), 5)

#intercept co-ordinates main line

    if(a1 != 0 and x1 == 0 ):
        slope_line_left = 0.009
        slope_line_right = float(b2-b1)/float(a2-a1)
    elif(x1 != 0 and a1 == 0):
        slope_line_left = float(y2-y1)/float(x2-x1)
        slope_line_right = 0.009
    else:
        slope_line_left = float(y2-y1)/float(x2-x1)
        slope_line_right = float(b2-b1)/float(a2-a1)



    # -------------------------------------------------------------------------------------------------------------------
    h1,k1,p1,q1 = 591,441,750,441
    h2,k2,p2,q2 = 194,441,374,441

    #refrence line (FIXED FOR ALL THE TIME)
    h1,k1,p1,q1 = 591,441,750,441
    h2,k2,p2,q2 = 194,441,374,441

    #turing line equations
    cv2.line(image, (h1,k1), (p1,q1), (0,0,255), 5)
    cv2.line(image, (h2,k2), (p2,q2), (0,0,255), 5)

    
#prtoblem in null array in singular matrix
    #solving for left lane line 
    a = np.array([[1,-1*slope_line_left],[1,0]])
    b = np.array([(y1-(slope_line_left*x1)),q2])
    x = np.linalg.solve(a, b)
    print 'left intercept point: '+str(x)

    #solving for the right lane line
    w = np.array([[1,-1*slope_line_right],[1,0]])
    s = np.array([(b1-(slope_line_right*a1)),q2])
    d = np.linalg.solve(w,s)
    print 'right intercept point: '+str(d)

    s,t = int(d[1]),int(d[0])

    w , h = new.shape[:2]
    e,r = int(x[1]),int(x[0])
    cv2.line(new,(e,r), (w,0), (0,0,0), 2)
    cv2.line(new,(s,t), (w,0), (0,0,0), 2)
    
    filename_right = 'distance_right.json'
    filenme_left = 'diatance_left.json'
    c_l = []
    c_r = []
    c_l.append(e)
    c_r.append(s)
    with open(filename_right,'a') as f_object:
        json.dump(c_r,f_object)

    with open(filenme_left,'a') as f_object:
        json.dump(c_l,f_object)
            
    #distance from the mid to the extreem left
   
            
    def ref_mid_points():
        right_mid_x = float(p1-h1)
        right_mid_y = 441
        left_mid_x = float(p2-h2)
        left_mid_y = 441
        cv2.putText(new, 'left refrence'.upper(), (w-100,200), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)

        if((x[1]-300)<0 ):
            print"turn left".upper()
            cv2.putText(new, 'left'.upper(), (w,100), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)
        elif((x[1]-300 )>0):
            print"turn right".upper()
            cv2.putText(new, 'right'.upper(), (w,100), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)
        elif(s > 500):
            #cv2.putText(new, 'no line'.upper(), (w,100), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)
            pass

            
    def turn():
        pass
 #------------------------------------------------------------------------------       
    ref_mid_points()
    turn()
#-------------------------------------------------------------------------------    
    
    #for distance from left
    def intercept_distance():
        r = []
        l = []


        l_distance = math.sqrt(pow(h2-x[1],2)+pow(k2-x[0],2))
        l.append(l_distance)
        if(l_distance<200):
            print 'distance from left: '.upper()+str(l_distance)    
        else:
            print 'line not found'

        r_distance = math.sqrt(pow(h1-d[1],2)+pow(k1-d[0],2))
        r.append(r_distance)
        if(r_distance<200):
            print 'distance from right: '.upper()+str(r_distance)    
        else:
            print 'line not found'

    cars = car_classifier.detectMultiScale(new, 1.4, 2)    
    for (x,y,w,h) in cars:
        cv2.rectangle(new, (x, y), (x+w, y+h), (0, 255, 255), 2)
        
    return new

In [23]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [19]:
yellow_output = 'submit-solidYellowLeftTestx2.mp4'
clip2 = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

nothing
nothing right side empty
[[570 365 758 497]
 [569 358 775 497]
 [356 391 415 355]]
right
[570 365 758 497]
[569 358 775 497]
left
[356 391 415 355]
new list left
[array([356, 391, 415, 355])]
new list right
[array([569, 358, 775, 497]), array([570, 365, 758, 497])]
new single line list left: [356 391 415 355]
new single line list right: [569 358 775 497]
left intercept point: [ 441.          274.05555556]
right intercept point: [ 441.          692.00719424]
TURN LEFT
[MoviePy] >>>> Building video submit-solidYellowLeftTestx2.mp4
[MoviePy] Writing video submit-solidYellowLeftTestx2.mp4


  0%|          | 0/682 [00:00<?, ?it/s]

nothing
nothing right side empty
[[563 359 767 496]
 [563 360 758 497]]
right
[563 359 767 496]
[563 360 758 497]
left
new list left
[0, 0, 0, 0]
new list right
[array([563, 359, 767, 496]), array([563, 359, 767, 496])]
new single line list left: 0
new single line list right: [563 359 767 496]
left intercept point: [   441.  49000.]
right intercept point: [ 441.          685.10218978]
TURN RIGHT


  0%|          | 1/682 [00:00<01:42,  6.62it/s]

nothing
nothing right side empty
[[567 362 814 535]
 [567 361 831 532]
 [352 394 415 355]
 [570 364 831 533]
 [568 364 812 535]]
right
[567 362 814 535]
[567 361 831 532]
[570 364 831 533]
[568 364 812 535]
left
[352 394 415 355]
new list left
[array([352, 394, 415, 355])]
new list right
[array([567, 361, 831, 532]), array([567, 361, 831, 532]), array([570, 364, 831, 533]), array([567, 362, 814, 535])]
new single line list left: [352 394 415 355]
new single line list right: [567 361 831 532]
left intercept point: [ 441.          276.07692308]
right intercept point: [ 441.          690.50877193]
TURN LEFT


...

nothing
nothing right side empty
[[179 538 422 355]
 [285 449 415 355]
 [551 355 781 498]
 [178 538 421 355]
 [559 355 798 498]
 [300 437 326 418]]
right
[551 355 781 498]
[559 355 798 498]
left
[179 538 422 355]
[285 449 415 355]
[178 538 421 355]
[300 437 326 418]
new list left
[array([179, 538, 422, 355]), array([178, 538, 421, 355]), array([179, 538, 422, 355])]
new list right
[array([559, 355, 798, 498]), array([551, 355, 781, 498])]
new single line list left: [179 538 422 355]
new single line list right: [559 355 798 498]
left intercept point: [ 441.          307.80327869]
right intercept point: [ 441.          702.73426573]
TURN RIGHT


100%|█████████▉| 680/682 [01:52<00:00,  5.97it/s]

nothing
nothing right side empty
[[178 538 422 355]
 [306 432 416 355]
 [553 355 674 430]
 [203 519 420 355]
 [559 355 683 429]
 [300 437 382 379]
 [179 539 352 408]]
right
[553 355 674 430]
[559 355 683 429]
left
[178 538 422 355]
[306 432 416 355]
[203 519 420 355]
[300 437 382 379]
[179 539 352 408]
new list left
[array([178, 538, 422, 355]), array([203, 519, 420, 355]), array([178, 538, 422, 355])]
new list right
[array([559, 355, 683, 429]), array([553, 355, 674, 430])]
new single line list left: [178 538 422 355]
new single line list right: [559 355 683 429]
left intercept point: [ 441.          307.33333333]
right intercept point: [ 441.          703.10810811]
TURN RIGHT


100%|█████████▉| 681/682 [01:52<00:00,  6.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: submit-solidYellowLeftTestx2.mp4 

Wall time: 1min 54s


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))